In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import payments_manager as pm
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
#pm.help()
#pm.reset()
pm.init() #debug=True)
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
#cr_cp.info()
#fe_cp.info()

df = pm.df('df_jo')
df_jall = pm.df('df_jall')
#df_jo.info()
#df_jall.info()

df = pm.sort("df_jo", ["id_cr"]).reset_index()
df = df.drop(columns=['index'])
#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df.copy()
#display(df_jo_cp)

In [4]:
df = pm.df('df_jall')
#display(df)
#display(df[~df.Mes_created_at.isna()])
df.info()
df['to_recive'] = df.cr_received_date-df.created_at
df['to_reimbur'] = df.reimbursement_date-df.created_at
df['to_end'] = df.reimbursement_date-df.money_back_date
df['to_send'] = df.send_at-df.created_at

cr_id = ['id_cr','user_id','amount','fee','stat_cr','stat_fe','transfer_type','type',
            'to_recive','to_reimbur','to_end','to_send',
            'created_at','reimbursement_date','cr_received_date','send_at','money_back_date',
            'paid_at','charge_moment','moderated_at'] # ,'id_y','from_date','to_date','reason','recovery_status', 'cash_request_id'

# print("Casos segun Cash Request ID")
# for id in ([ 23534]): # 20108, 20104, 20112,
#     df_t = df[df['id_cr'] == id].reset_index()
#     print(f"Cash Request ID: {id}")
#     display(df_t[cr_id].sort_values('created_at'))

print("Casos segun Cash User ID")
for id in ([ 63894,18730,10116,21465, 99000262]):
    df_t = df[df['user_id'] == id].reset_index()
    print(f"User ID: {id}")
    display(df_t[cr_id].sort_values('created_at'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32094 entries, 0 to 32093
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_cr               32094 non-null  int64         
 1   amount              32094 non-null  float64       
 2   stat_cr             32094 non-null  object        
 3   created_at          32094 non-null  datetime64[ns]
 4   updated_at_x        32094 non-null  datetime64[ns]
 5   user_id             32094 non-null  int64         
 6   active              32094 non-null  int64         
 7   moderated_at        21759 non-null  datetime64[ns]
 8   reimbursement_date  32094 non-null  datetime64[ns]
 9   cr_received_date    24149 non-null  datetime64[ns]
 10  money_back_date     23917 non-null  datetime64[ns]
 11  transfer_type       32094 non-null  object        
 12  send_at             22678 non-null  datetime64[ns]
 13  recovery_status     7200 non-null   object    

,id_cr,user_id,amount,fee,stat_cr,stat_fe,transfer_type,type,to_recive,to_reimbur,to_end,to_send,created_at,reimbursement_date,cr_received_date,send_at,money_back_date,paid_at,charge_moment,moderated_at
0,20108,63894,100.0,5.0,money_back,accepted,instant,instant_payment,0 days 12:57:05.928453,27 days 23:57:05.928453,-1 days +15:32:21.086619,0 days 00:00:31.417249,2020-10-09 11:02:54.071547,2020-11-06 11:00:00,2020-10-10,2020-10-09 11:03:25.488796,2020-11-06 19:27:38.913381,2020-11-06 19:27:38.941414,after,NaT
1,20108,63894,100.0,5.0,money_back,cancelled,instant,postpone,0 days 12:57:05.928453,27 days 23:57:05.928453,-1 days +15:32:21.086619,0 days 00:00:31.417249,2020-10-09 11:02:54.071547,2020-11-06 11:00:00,2020-10-10,2020-10-09 11:03:25.488796,2020-11-06 19:27:38.913381,NaT,after,NaT
2,20108,63894,100.0,5.0,money_back,cancelled,instant,postpone,0 days 12:57:05.928453,27 days 23:57:05.928453,-1 days +15:32:21.086619,0 days 00:00:31.417249,2020-10-09 11:02:54.071547,2020-11-06 11:00:00,2020-10-10,2020-10-09 11:03:25.488796,2020-11-06 19:27:38.913381,NaT,after,NaT


User ID: 18730


,id_cr,user_id,amount,fee,stat_cr,stat_fe,transfer_type,type,to_recive,to_reimbur,to_end,to_send,created_at,reimbursement_date,cr_received_date,send_at,money_back_date,paid_at,charge_moment,moderated_at
1,4187,18730,100.0,NaN,money_back,NaN,regular,NaN,3 days 11:55:21.512398,17 days 09:55:21.512398,-1 days +00:00:00,NaT,2020-06-19 12:04:38.487602,2020-07-06 22:00:00.000,2020-06-23,NaT,2020-07-07 22:00:00.000000,NaT,NaN,2020-06-19 13:05:01.750011
2,8416,18730,100.0,NaN,money_back,NaN,regular,NaN,4 days 16:05:05.642987,20 days 14:05:05.642987,0 days 00:00:00,NaT,2020-07-16 07:54:54.357013,2020-08-05 22:00:00.000,2020-07-21,NaT,2020-08-05 22:00:00.000000,NaT,NaN,2020-07-17 10:55:36.753979
4,12099,18730,100.0,NaN,money_back,NaN,regular,NaN,8 days 00:54:27.180273,26 days 22:54:27.180273,-1 days +02:44:22.134158,6 days 23:59:59.999323,2020-08-11 23:05:32.819727,2020-09-07 22:00:00.000,2020-08-20,2020-08-18 23:05:32.819050,2020-09-08 19:15:37.865842,NaT,NaN,NaT
3,15336,18730,100.0,NaN,money_back,NaN,regular,NaN,8 days 11:36:50.046060,27 days 09:36:50.046060,-1 days +02:05:27.333983,6 days 23:59:59.953783,2020-09-09 12:23:09.953940,2020-10-06 22:00:00.000,2020-09-18,2020-09-16 12:23:09.907723,2020-10-07 19:54:32.666017,NaT,NaN,NaT
0,20104,18730,100.0,5.0,money_back,cancelled,regular,postpone,10 days 13:01:11.680670,15 days 00:01:08.332670,-5 days +11:59:56.652000,6 days 23:59:59.996975,2020-10-09 10:58:48.319330,2020-10-24 10:59:56.652,2020-10-20,2020-10-16 10:58:48.316305,2020-10-28 23:00:00.000000,NaT,before,NaT


User ID: 10116


,id_cr,user_id,amount,fee,stat_cr,stat_fe,transfer_type,type,to_recive,to_reimbur,to_end,to_send,created_at,reimbursement_date,cr_received_date,send_at,money_back_date,paid_at,charge_moment,moderated_at
1,2262,10116,100.0,NaN,rejected,NaN,regular,NaN,NaT,9 days 23:54:55.516952,NaT,NaT,2020-05-28 22:05:04.483048,2020-06-07 22:00:00,NaT,NaT,NaT,NaT,NaN,2020-05-29 08:12:30.061672
3,2661,10116,100.0,NaN,money_back,NaN,regular,NaN,2 days 07:33:33.969127,31 days 05:33:33.969127,0 days 00:00:00,NaT,2020-06-06 16:26:26.030873,2020-07-07 22:00:00,2020-06-09,NaT,2020-07-07 22:00:00.000000,NaT,NaN,2020-06-06 17:50:34.061784
2,8571,10116,100.0,NaN,rejected,NaN,regular,NaN,NaT,19 days 23:40:36.808061,NaT,NaT,2020-07-16 22:19:23.191939,2020-08-05 22:00:00,NaT,NaT,NaT,NaT,NaN,2020-07-17 15:19:10.535343
4,9689,10116,100.0,NaN,money_back,NaN,regular,NaN,2 days 09:05:27.415380,11 days 07:05:27.415380,0 days 00:00:00,6 days 23:59:59.999640,2020-07-25 14:54:32.584620,2020-08-05 22:00:00,2020-07-28,2020-08-01 14:54:32.584260,2020-08-05 22:00:00.000000,NaT,NaN,2020-07-25 18:22:07.431781
6,11710,10116,50.0,5.0,money_back,accepted,instant,instant_payment,1 days 10:47:06.511345,25 days 08:47:06.511345,-1 days +02:09:33.756680,6 days 23:59:59.999598,2020-08-10 13:12:53.488655,2020-09-04 22:00:00,2020-08-12,2020-08-17 13:12:53.488253,2020-09-05 19:50:26.243320,2020-09-05 19:50:26.278501,after,2020-08-10 16:06:32.371677
5,16298,10116,100.0,5.0,money_back,accepted,instant,instant_payment,1 days 04:48:30.355002,20 days 02:48:30.355002,-1 days +02:26:06.846194,6 days 23:59:59.951342,2020-09-15 19:11:29.644998,2020-10-05 22:00:00,2020-09-17,2020-09-22 19:11:29.596340,2020-10-06 19:33:53.153806,2020-10-06 19:33:53.180290,after,2020-09-16 07:55:34.095459
0,20112,10116,100.0,5.0,money_back,accepted,instant,instant_payment,0 days 12:47:24.809622,27 days 10:47:24.809622,-1 days +01:24:56.222555,0 days 00:00:06.659528,2020-10-09 11:12:35.190378,2020-11-05 22:00:00,2020-10-10,2020-10-09 11:12:41.849906,2020-11-06 20:35:03.777445,2020-11-06 20:35:03.804527,after,NaT


User ID: 21465


,id_cr,user_id,amount,fee,stat_cr,stat_fe,transfer_type,type,to_recive,to_reimbur,to_end,to_send,created_at,reimbursement_date,cr_received_date,send_at,money_back_date,paid_at,charge_moment,moderated_at
1,16889,21465,50.0,NaN,rejected,NaN,regular,NaN,NaT,35 days 06:52:21.023770,NaT,6 days 23:59:59.978356,2020-09-19 15:07:38.976230,2020-10-24 22:00:00,NaT,2020-09-26 15:07:38.954586,NaT,NaT,NaN,2020-09-21 09:17:04.343374
0,23534,21465,25.0,5.0,money_back,accepted,instant,instant_payment,NaT,18 days 08:30:33.836073,4 days 16:34:38.154521,0 days 00:01:00.714598,2020-10-23 15:20:26.163927,2020-11-10 23:51:00,NaT,2020-10-23 15:21:26.878525,2020-11-06 07:16:21.845479,2020-11-06 07:16:22.014422,after,NaT


User ID: 99000262


,id_cr,user_id,amount,fee,stat_cr,stat_fe,transfer_type,type,to_recive,to_reimbur,to_end,to_send,created_at,reimbursement_date,cr_received_date,send_at,money_back_date,paid_at,charge_moment,moderated_at
0,280,99000262,100.0,NaN,money_back,NaN,regular,NaN,2 days 07:58:21.243015,26 days 06:58:21.243015,-312 days +12:28:05.401407,NaT,2019-12-21 16:01:38.756985,2020-01-16 23:00:00,2019-12-24,NaT,2020-11-23 10:31:54.598593,NaT,NaN,2019-12-22 13:48:26
